In [4]:
!pip install pandas sklearn xgboost hyperopt

In [5]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.pipeline import Pipeline
import pickle
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import warnings
from sklearn.feature_selection import SelectKBest, chi2
warnings.filterwarnings('ignore')


## Generate baseline model

In [6]:
df = pd.read_csv(r"https://raw.githubusercontent.com/medinaltbx/G6_DP3/master/data/input/merged_data/train/merged_train.csv",sep=';')
df.drop(["customerid"],axis=1,inplace=True)
X, y = df.drop(["good_bad_flag"],axis=1), df['good_bad_flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

baseline_model = xgb.XGBClassifier()
baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_test)
baseline_accuracy = accuracy_score(y_test, y_pred)
print("BASELINE ACCURACY: ", baseline_accuracy)

BASELINE ACCURACY:  0.7835365853658537


## Generate model with base variables + Grid Search

In [7]:
df = pd.read_csv(rf"https://raw.githubusercontent.com/medinaltbx/G6_DP3/master/data/input/merged_data/train/merged_train.csv",sep=';')
df.drop(["customerid"],axis=1,inplace=True)
X, y = df.drop(["good_bad_flag"],axis=1), df['good_bad_flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pipe = Pipeline([('fs', SelectKBest()),('clf', xgb.XGBClassifier(objective='binary:logistic'))])

# Define our search space for grid search
search_space = [
  {
    'clf__n_estimators': [100, 200, 300],
    'clf__learning_rate': [0.01, 0.1],
    'clf__max_depth': range(5, 10),
    'clf__colsample_bytree': [i/10.0 for i in range(1, 3)],
    'clf__gamma': [i/10.0 for i in range(3)],
    'fs__score_func': [chi2],
  }
]
# Define cross validation
kfold = KFold(n_splits=3)
# AUC and accuracy as score
scoring = {'AUC':'roc_auc', 'Accuracy':make_scorer(accuracy_score)}
# Define grid search
grid = GridSearchCV(
  pipe,
  param_grid=search_space,
  cv=kfold,
  scoring=scoring,
  refit='AUC',
  verbose=1,
  n_jobs=-1
)
# Fit grid search
complete_model = grid.fit(X_train, y_train)
predict_complete = complete_model.predict(X_test)
complete_accuracy = accuracy_score(y_test, predict_complete)
print('Best AUC Score: {}'.format(complete_model.best_score_))
print('Accuracy: {}'.format(complete_accuracy))

Fitting 3 folds for each of 180 candidates, totalling 540 fits
Best AUC Score: 0.6856376300878894
Accuracy: 0.7713414634146342


## Generate model with cluster feature + Grid Search

In [8]:
df = pd.read_csv(rf"https://raw.githubusercontent.com/medinaltbx/G6_DP3/master/data/input/merged_data/train/cluster_train.csv",sep=';')
df.drop(["customerid"],axis=1,inplace=True)
X, y = df.drop(["good_bad_flag"],axis=1), df['good_bad_flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pipe = Pipeline([('fs', SelectKBest()),('clf', xgb.XGBClassifier(objective='binary:logistic'))])

# Define our search space for grid search
search_space = [
  {
    'clf__n_estimators': [100, 200, 300],
    'clf__learning_rate': [0.01, 0.1],
    'clf__max_depth': range(10, 20),
    'clf__colsample_bytree': [i/10.0 for i in range(1, 3)],
    'clf__gamma': [i/10.0 for i in range(3)],
    'fs__score_func': [chi2],
  }
]
# Define cross validation
kfold = KFold(n_splits=3)
# AUC and accuracy as score
scoring = {'AUC':'roc_auc', 'Accuracy':make_scorer(accuracy_score)}
# Define grid search
grid = GridSearchCV(
  pipe,
  param_grid=search_space,
  cv=kfold,
  scoring=scoring,
  refit='AUC',
  verbose=1,
  n_jobs=-1
)
# Fit grid search
cluster_model = grid.fit(X_train, y_train)
predict_cluster = cluster_model.predict(X_test)
cluster_accuracy = accuracy_score(y_test, predict_cluster)
print('Best AUC Score: {}'.format(cluster_model.best_score_))
print('Accuracy: {}'.format(cluster_accuracy))

Fitting 3 folds for each of 180 candidates, totalling 540 fits
Best AUC Score: 0.6399587151476193
Accuracy: 0.7972560975609756


## Generate model with cluster top 10 features + Grid Search

In [9]:
df = pd.read_csv(rf"https://raw.githubusercontent.com/medinaltbx/G6_DP3/master/data/input/merged_data/train/top_10_train.csv",sep=';')
X, y = df.drop(["good_bad_flag"],axis=1), df['good_bad_flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pipe = Pipeline([('fs', SelectKBest()),('clf', xgb.XGBClassifier(objective='binary:logistic'))])

# Define our search space for grid search
search_space = [
  {
    'clf__n_estimators': [100, 200, 300],
    'clf__learning_rate': [0.01, 0.1],
    'clf__max_depth': range(5, 10),
    'clf__colsample_bytree': [i/10.0 for i in range(1, 3)],
    'clf__gamma': [i/10.0 for i in range(3)],
    'fs__score_func': [chi2],
  }
]
# Define cross validation
kfold = KFold(n_splits=3)
# AUC and accuracy as score
scoring = {'AUC':'roc_auc', 'Accuracy':make_scorer(accuracy_score)}
# Define grid search
grid = GridSearchCV(
  pipe,
  param_grid=search_space,
  cv=kfold,
  scoring=scoring,
  refit='AUC',
  verbose=1,
  n_jobs=-1
)
# Fit grid search
top_10_model = grid.fit(X_train, y_train)
predict_top_10 = top_10_model.predict(X_test)
top_10_accuracy = accuracy_score(y_test, predict_top_10)
print('Best AUC Score: {}'.format(top_10_model.best_score_))
print('Accuracy: {}'.format(top_10_acc))

Fitting 3 folds for each of 180 candidates, totalling 540 fits
Best AUC Score: 0.6656971296901795
Accuracy: 0.7774390243902439


# Model comparison

In [11]:
accuracies = [(baseline_accuracy, baseline_model), 
              (complete_accuracy, complete_model),
              (top_10_accuracy, top_10_model),
              (cluster_accuracy, cluster_model)]

best_model, max_acc = None, 0
for t in accuracies:
    if t[0] > max_acc:
        max_acc = t[0]
        best_model = t[1]

print("Best Model: ", best_model)
print("Best Accuracy: ", max_acc)

Best Model:  GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('fs', SelectKBest()),
                                       ('clf',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric=None,
                                                      gamma=None, gpu_id=None,
                                                      g